In [1]:
import json 

with open('data/train.json') as f:
    train_data = json.load(f)

In [2]:
train_sentences = []
train_labels= []
for i in range(len(train_data)):
    for j in train_data[i]['sentence']:
        train_sentences.append(j)

for k in range(len(train_data)):
    for l in train_data[k]['labels']:
        train_labels.append(l)

In [3]:
print(len(train_sentences))
print(len(train_labels))

912095
912095


In [4]:
vocab = {}
for i in range(len(train_sentences)):
    if train_sentences[i] in vocab:
        vocab[train_sentences[i]] += 1
    else:
        vocab[train_sentences[i]] = 1

In [5]:
known_vocab = {}
num_unk = 0

for w in vocab:
    if vocab[w] >= 3:
        known_vocab[w] = vocab[w]
    else:
        num_unk += vocab[w]

In [6]:
vocab_sorted = sorted(vocab.items(), key=lambda item: item[1], reverse=True)

In [7]:
43193 * 45

1943685

In [8]:

with open('output/vocab.txt', 'w') as vocab_file:
    vocab_file.write('<unk>' + '\t' + str(0) + '\t' + str(num_unk) + '\n')
    for i in range(len(vocab_sorted)):
        vocab_file.write(str(vocab_sorted[i][0]) + '\t' + str(i+1) + '\t' + str(vocab_sorted[i][1]) + '\n')

In [9]:
print('Size of vocab : ',len(vocab_sorted))
print('\nTotal occurrences of <unk> is : ',num_unk)

Size of vocab :  43193

Total occurrences of <unk> is :  32537


In [10]:
vocab_ls = list(known_vocab.keys())

for i in range(len(train_sentences)):
    if train_sentences[i] not in vocab_ls:
        train_sentences[i] = "<unk>"


In [11]:
known_words = list(known_vocab.keys())

In [12]:
sx = {}

for i in range(len(train_data)):
    for j in range(len(train_data[i]['sentence'])):
        #print(train_data[i]['sentence'][j])
        sent = train_data[i]['sentence']
        label = train_data[i]['labels']
        if str(sent[j]) + '||' + str(label[j]) in sx:
            sx[str(sent[j]) + '||' + str(label[j])] +=1
        else:
            sx[str(sent[j]) + '||' + str(label[j])] = 1




In [13]:
for i in range(len(train_data)):
    for j in range(len(train_data[i]['sentence'])):
        sent = train_data[i]['sentence']
        label = train_data[i]['labels']
        if sent[j]  not in known_words:
            tg = "<unk>"
            if str(tg) + '||' + str(label[j]) in sx:
                sx[str(tg) + '||' + str(label[j])] += 1
            else:
                sx[str(tg) + '||' + str(label[j])] = 1

In [14]:
sx['<unk>||NNS']

3242

In [15]:
ss = {}
for i in range(len(train_data)):
    for j in range(len(train_data[i]['labels'])):
        label = train_data[i]['labels']
        if j+1 < len(train_data[i]['labels']):
            if str(label[j+1]) + '||' + str(label[j]) in ss:
                ss[str(label[j+1]) + '||' + str(label[j])] += 1
            else:
                ss[str(label[j+1]) + '||' + str(label[j])] = 1


In [16]:
len(ss)

1351

In [17]:
# sxsx = {}
# for i in range(len(train_sentences)):
#     # print("word: ", train_sentences[i])
#     # print("label: ", train_labels[i])
#     # break
#     if train_sentences[i] in known_words:
#         if str(train_sentences[i]) + '|' + str(train_labels[i]) in sxsx:
#             sxsx[str(train_sentences[i]) + '|' + str(train_labels[i])] += 1
#         else: 
#             sxsx[str(train_sentences[i]) + '|' + str(train_labels[i])] = 1
#     else:
#         s = "<unk>"
#         if str(s) + '|' + str(train_labels[i]) in sxsx:
#             sxsx[str(s) + '|' + str(train_labels[i])] += 1
#         else: 
#             sxsx[str(s) + '|' + str(train_labels[i])] = 1


In [18]:
# len(sxsx)

In [19]:
# check = 0
# for i in range(len(train_sentences)):
#     if (train_sentences[i] not in vocab2.keys()) and (train_labels[i] == 'NNP'):
#         check +=1
# check

In [20]:
pos_count={}

for i in range(len(train_data)):
    for j in range(len(train_data[i]['labels'])):
        label = train_data[i]['labels']
        if label[j] in pos_count:
            pos_count[label[j]] += 1
        else:
            pos_count[label[j]] = 1

In [21]:
distinct_pos = list(pos_count.keys())

In [22]:
pos_count['NNP']

87608

In [23]:
emission = {}
transition = {}

for i in sx:
    emission[i] = sx[i] / pos_count[str(i).split('||')[1]]

for j in ss:
    transition[j] = ss[j] / pos_count[str(j).split('||')[1]]

In [24]:
len(emission)

50320

In [25]:
len(transition)

1351

In [26]:
len(train_data)

38218

In [27]:
pr = {}
for i in range(len(train_data)):
    if train_data[i]['labels'][0] in pr:
        pr[train_data[i]['labels'][0]] += 1
    else:
        pr[train_data[i]['labels'][0]] = 1

In [28]:
prior = {}

for k in pr.keys():
    prior[k] = pr[k]/len(train_data)

 

In [29]:
hmm = {"initial: ":prior,"transition":transition, "emission":emission}
json_object = json.dumps(hmm, indent=2)
with open('output/hmm.json', 'w') as op:
    op.write(json_object)

In [30]:
0.026599146200245425 * pos_count['NNS']

1539.0

In [31]:
sx['<unk>||NNS']

3242

In [32]:
vocab['Huricane']

1

In [33]:
mlist = []

def greedy(sentence):
    postags = []
    sent = sentence[0]
    if sent not in known_words:
        sent = "<unk>"
    maxp = 0
    p0 = 'UNK'

    for p in distinct_pos:
        try:
            temp = emission[sent + '||' + p] * prior[p]
            if temp>maxp:
                maxp = temp
                p0 = p
        except:
            pass
    postags.append(p0)
    

    for i in range(1, len(sentence)):
        sent = sentence[i]
        if sent not in known_words:
            sent = "<unk>"
        max_p = 0
        pi = 'UNK'

        for p in distinct_pos:
            try:
                temp = emission[sent + '||' + p] * transition[p + '||' + postags[-1]]
                if temp > max_p:
                    max_p = temp 
                    pi = p
            except:
                pass
        postags.append(pi)

    mlist.append(postags)
    return postags


with open('data/dev.json') as data:
    dev_data = json.load(data)
glist=[]
for i in range(len(dev_data)):
    glist.append(greedy(dev_data[i]['sentence']))

In [34]:
pred_list = []

for i in range(len(glist)):
    for j in glist[i]:
        pred_list.append(j)


In [35]:
target_list = []
for i in range(len(dev_data)):
    for j in range(len(dev_data[i]['labels'])):
        target_list.append(dev_data[i]['labels'][j])


In [36]:
len(target_list)

131768

In [37]:
len(pred_list)

131768

In [38]:
correct = 0
for i in range(len(target_list)):
    if target_list[i] == pred_list[i]:
        correct+=1
correct

122119

In [39]:
(correct/131768)*100

92.67728128225366

In [40]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(target_list, pred_list)
print(acc)

0.9267728128225365


In [41]:
def viterbi(vsentence):
    s = {i:{} for i in range(len(vsentence))}
    initialpos = {i:{} for i in range(len(vsentence))}
    sent = vsentence[0]
    if sent not in known_words:
        sent = "<unk>"
    
    for p in distinct_pos:
        if p in prior:
            try:
                s[0][p] = emission[sent + '||' + p] * prior[p]
            except:
                s[0][p] =0
    for p in s[0].keys():
        initialpos[0][p] = 'NNP'

    for i in range(1, len(vsentence)):
        sent = vsentence[i]
        if sent not in known_words:
            sent = "<unk>"
            
        for p in s[i-1].keys():
            for pos_i in distinct_pos:
                if pos_i + '||' + p in transition:
                    if pos_i in s[i]:
                        try:
                            temp = s[i-1][p] * transition[pos_i + '||' + p] * emission[sent + '||' + pos_i]
                            if temp > s[i][pos_i]:
                                s[i][pos_i] = temp
                                initialpos[i][pos_i] = p
                        except:
                            pass
                    else:
                        try:
                            s[i][pos_i] = s[i-1][p] * transition[pos_i + '||' + p] * emission[sent + '||' + pos_i]
                            initialpos[i][pos_i] = p
                        except:

                            s[i][pos_i] = 0

    
    s_pred = []

    maxp = max(s[len(vsentence)-1].values())

    maxp_index = list(s[len(vsentence)-1].values()).index(maxp)

    pos_maxp = list(s[len(vsentence)-1].keys())[maxp_index]
    s_pred.append(pos_maxp)

    for q in range(len(vsentence)-1,0,-1):
        try:
            pos_maxp = initialpos[q][pos_maxp]
            s_pred.append(pos_maxp)
        except:
            s_pred.append('UNK')
    
    reversed_s_pred=[]
    for i  in range(len(s_pred)-1, -1, -1):
        reversed_s_pred.append(s_pred[i])

    return reversed_s_pred



with open('data/dev.json') as data:
    dev_data = json.load(data)

v_pred_list = []
for i in range(len(dev_data)):
    v_pred_list.append(viterbi(dev_data[i]['sentence']))



In [42]:
vpred = []
for i in range(len(v_pred_list)):
    for j in v_pred_list[i]:
        vpred.append(j)

In [43]:
vcorrect = 0
for i in range(len(target_list)):
    if target_list[i] == vpred[i]:
        vcorrect+=1
print(vcorrect)

124344


In [44]:
(vcorrect/131768)*100

94.36585513933579

In [45]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(target_list, vpred)
print(acc)

0.943658551393358


In [46]:
with open('data/test.json') as data:
    test_data = json.load(data)

viterbi_pred_list = []
for i in range(len(test_data)):
    #print(test_data[i]['sentence'])
    viterbi_pred_list.append(viterbi(test_data[i]['sentence']))



In [47]:
len(viterbi_pred_list)

5462

In [48]:
len(test_data)

5462

In [49]:
viterbil = []


for i in range(len(viterbi_pred_list)):
    viterbidict = {}
    viterbidict["index"] = i
    viterbidict["sentence"] = test_data[i]['sentence'] 
    viterbidict["labels"] = viterbi_pred_list[i]
    viterbil.append(viterbidict)


In [50]:
len(viterbil)

5462

In [51]:
viterbijson = json.dumps(viterbil, indent=2)
with open('output/viterbi.json','w') as op:
    op.write(viterbijson)

In [52]:
with open('data/test.json') as data:
    test_data = json.load(data)

greedy_pred_list = []
for i in range(len(test_data)):
    #print(test_data[i]['sentence'])
    greedy_pred_list.append(greedy(test_data[i]['sentence']))




In [53]:
greedyl = []

for i in range(len(greedy_pred_list)):
    greedydict = {}
    greedydict["index"] = i
    greedydict["sentence"] = test_data[i]['sentence'] 
    greedydict["labels"] = greedy_pred_list[i]
    greedyl.append(greedydict)


In [54]:
z = {}
z['index'] = 1
z['sentence'] = ['a', 'b', 'c']
z['label'] = ['x','y', 'z']

In [55]:
ztest = []
ztest.append(z)
ztest

[{'index': 1, 'sentence': ['a', 'b', 'c'], 'label': ['x', 'y', 'z']}]

In [56]:
len(greedyl)

5462

In [57]:
len(greedy_pred_list)

5462

In [58]:
greedyjson = json.dumps(greedyl, indent=2)
with open('output/greedy.json','w') as op:
    op.write(greedyjson)